### 실시간 인트라데이 데이터 수집
 - 5분 간격의 실시간 주가 데이터
 - AAPL, GOOGL, TSLA 종목 데이터
 - 시간대별 거래량 통계
 
### 기술적 지표 데이터
 - RSI (Relative Strength Index) 지표
 - 재사용 가능한 함수 형태로 구현
 - 지표별 통계 분석
 
### 데이터 관리
 - 타임스탬프가 포함된 파일명으로 저장
 - API 호출 제한 준수
 - 오류 처리 및 데이터 검증

In [1]:
# Alpha Vantage 실시간 인트라데이 주가 데이터
import os
import pandas as pd
import requests
import time
from datetime import datetime, timedelta

#API_KEY = os.environ.get('ALPHAVANTAGE_API_KEY')
API_KEY = 'OACKEXPDGZWGF7KE'
BASE = 'https://www.alphavantage.co/query'

# 실시간 인트라데이 데이터 설정
symbols = ['AAPL', 'GOOGL', 'TSLA']
params_template = {
    'function': 'TIME_SERIES_INTRADAY',
    'interval': '5min',  # 1min, 5min, 15min, 30min, 60min 중 선택
    'outputsize': 'compact',  # compact(최근 100개) 또는 full
    'datatype': 'json'
}

os.makedirs('data/collected', exist_ok=True)


In [2]:
# 실시간 인트라데이 데이터 수집
frames = []
if API_KEY:
    for sym in symbols:
        try:
            params = params_template | {'symbol': sym, 'apikey': API_KEY}
            r = requests.get(BASE, params=params, timeout=30)
            r.raise_for_status()
            js = r.json()

            # API 응답 확인
            time_series_key = f'Time Series ({params["interval"]})'
            ts = js.get(time_series_key, {})

            if not ts:
                print(f"{sym}: 데이터 없음")
                continue

            # DataFrame 생성
            df = pd.DataFrame(ts).T.reset_index().rename(columns={'index': 'timestamp'})

            # 데이터 타입 변환
            numeric_cols = ['1. open', '2. high', '3. low', '4. close', '5. volume']
            for col in numeric_cols:
                if col in df.columns:
                    df[col] = pd.to_numeric(df[col], errors='coerce')

            df['symbol'] = sym
            df['interval'] = params['interval']
            frames.append(df)

            # API 호출 제한 준수 (초당 5회, 분당 500회)
            time.sleep(0.25)

        except Exception as e:
            print(f"{sym}: 오류 발생 - {str(e)}")
            continue

intraday_df = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()


In [ ]:
# 최근 5개 데이터 확인
if not intraday_df.empty:
    print(f"총 데이터 개수: {len(intraday_df)}")
    print("\n최근 5개 데이터:")
    display(intraday_df.head())

    # 시간대별 통계
    intraday_df['datetime'] = pd.to_datetime(intraday_df['timestamp'])
    intraday_df['hour'] = intraday_df['datetime'].dt.hour

    print("\n시간대별 평균 거래량:")
    hourly_volume = intraday_df.groupby('hour')['5. volume'].mean()
    display(hourly_volume)
else:
    print("데이터가 없습니다 (API_KEY 확인 필요)")


In [ ]:
# CSV 파일로 저장
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
output_file = f'data/collected/alpha_vantage_intraday_{timestamp}.csv'

if not intraday_df.empty:
    # 시간 순서대로 정렬
    intraday_df = intraday_df.sort_values('timestamp', ascending=False)
    intraday_df.to_csv(output_file, index=False)
    print(f"저장 완료: {output_file}")
else:
    print("저장할 데이터가 없습니다")


In [5]:
# Alpha Vantage 기술적 지표 데이터 (RSI)
def get_technical_indicator(symbol, function='RSI', interval='daily', time_period=14):
    """기술적 지표 데이터 가져오기"""
    params = {
        'function': f'{function}',
        'symbol': symbol,
        'interval': interval,
        'time_period': time_period,
        'series_type': 'close',
        'datatype': 'json',
        'apikey': API_KEY
    }

    try:
        r = requests.get(BASE, params=params, timeout=30)
        r.raise_for_status()
        js = r.json()

        # Technical Analysis 키 찾기
        tech_key = f'Technical Analysis: {function}'
        ts = js.get(tech_key, {})

        if ts:
            df = pd.DataFrame(ts).T.reset_index().rename(columns={'index': 'date'})
            df['symbol'] = symbol
            df['indicator'] = function
            return df
    except Exception as e:
        print(f"{symbol} {function}: 오류 - {str(e)}")

    return pd.DataFrame()

# RSI 지표 데이터 수집 예시
rsi_frames = []
for sym in symbols[:2]:  # 상위 2개 종목만
    df = get_technical_indicator(sym, 'RSI', 'daily', 14)
    if not df.empty:
        rsi_frames.append(df)
    time.sleep(0.25)  # API 제한 준수

rsi_df = pd.concat(rsi_frames, ignore_index=True) if rsi_frames else pd.DataFrame()


In [ ]:
# RSI 데이터 확인
if not rsi_df.empty:
    print("RSI 데이터 (최근 10개):")
    display(rsi_df.head(10))

    # RSI를 숫자로 변환
    rsi_df['RSI'] = pd.to_numeric(rsi_df['RSI'], errors='coerce')

    print("\nRSI 통계:")
    print(rsi_df.groupby('symbol')['RSI'].describe())
else:
    print("RSI 데이터가 없습니다")
